## Desafio

### Importação pandas

In [4]:
import pandas as pd

df = pd.read_csv('dataset.csv')

print(df.head())

  Genero_Usuario  Idade_Usuario   Bike  Estacao_Aluguel Data_Aluguel  \
0              M             44   4357              442   01/02/2020   
1              M             22  12083               66   01/02/2020   
2              M             29  11562              331   01/02/2020   
3              M             27  10206              164   01/02/2020   
4              M             27  10101              120   01/02/2020   

  Hora_Aluguel  Estacao_Chegada Data_Chegada Hora_Chegada  
0      0:00:38              116   01/02/2020      0:35:17  
1      0:00:53               37   01/02/2020      0:06:23  
2      0:00:55              341   01/02/2020      0:26:47  
3      0:01:18               35   01/02/2020      0:16:51  
4      0:01:18               47   01/02/2020      0:12:39  


### 1.Quais são as Top 5 estações com maior número de aluguel de bikes?

In [ ]:
df['contador'] = 1

estacoes_agrupadas = df.groupby('Estacao_Aluguel')['contador'].sum().reset_index()

top_5_estacoes = estacoes_agrupadas.sort_values(by='contador', ascending=False).head(5)
top_5_estacoes.rename(columns={'contador': 'QTD'}, inplace=True)

print(top_5_estacoes)

# top_5_estacoes.to_csv('top_5_estacoes.csv',index=False)


    Estacao_Aluguel   QTD
0                 1  6311
188              27  6241
190             271  5265
438              64  4834
344              41  4633


### 2.Quais são as Top 5 rotas, com base na estação inicial e final, e a média de duração de cada aluguel? 

In [ ]:
df['Hora_Aluguel'] = pd.to_datetime(df['Hora_Aluguel'], format='%H:%M:%S')
df['Hora_Chegada'] = pd.to_datetime(df['Hora_Chegada'], format='%H:%M:%S')

df['Duracao_Minutos'] = (df['Hora_Chegada'] - df['Hora_Aluguel']).dt.total_seconds() / 60

df['Estacao_Aluguel'] = df['Estacao_Aluguel'].astype(str)
df['Estacao_Chegada'] = df['Estacao_Chegada'].astype(str)

df['Rota'] = df['Estacao_Aluguel'] + ' -> ' + df['Estacao_Chegada']

rotas_agrupadas = df.groupby('Rota').agg(
    total_alugueis=('Rota', 'count'),
    media_duracao_minutos=('Duracao_Minutos', 'mean')
).reset_index()
rotas_agrupadas['media_duracao_minutos'] = rotas_agrupadas['media_duracao_minutos'].round(2)


top_5_rotas = rotas_agrupadas.sort_values(by='total_alugueis', ascending=False).head(5)

print(top_5_rotas)

# top_5_rotas.to_csv('top_5_rotas.csv',index=False)



             Rota  total_alugueis  media_duracao_minutos
56619    33 -> 33             376                  27.00
20184     18 -> 1             319                   5.58
77073  449 -> 449             304                   7.84
28893  211 -> 217             303                   3.54
27868  208 -> 206             298                   9.14


### 3.Quem aluga mais bikes, homens ou mulheres? Qual o tempo médio de aluguel de bikes? 

In [ ]:
mapeamento_genero = {'M': 'Masculino', 'F': 'Feminino'}
genero_mais_aluga = df.groupby('Genero_Usuario').agg(
    total=('Genero_Usuario', 'count'),
    media_duracao_minutos=('Duracao_Minutos', 'mean')
).reset_index()

genero_mais_aluga['media_duracao_minutos'] = rotas_agrupadas['media_duracao_minutos'].round(2)

print(genero_mais_aluga)
genero_mais_alugueis = genero_mais_aluga.loc[genero_mais_aluga['total'].idxmax()]
genero_nome = mapeamento_genero.get(genero_mais_alugueis['Genero_Usuario'], "Desconhecido")

mensagem = (
    f"O gênero que mais aluga bikes é {genero_nome} "
    f"com um total de {genero_mais_alugueis['total']} aluguéis. "
    f"O tempo médio de aluguel é {genero_mais_alugueis['media_duracao_minutos']:.2f} minutos."
)

print(genero_mais_alugueis)
# genero_mais_aluga.to_csv('genero_mais_aluga.csv',index=False)


  Genero_Usuario   total  media_duracao_minutos
0              F  175224                  16.88
1              M  511103                   8.79
Genero_Usuario                M
total                    511103
media_duracao_minutos      8.79
Name: 1, dtype: object


### 4.Qual faixa etária aluga mais bikes? Qual o tempo médio de aluguel de bikes? 

In [ ]:
faixas_definidas = [15, 17, 24, 30, 39, 50, 60, float('inf')]

faixas_texto = [
    '15 - 17 anos',
    '18 - 24 anos',
    '25 - 30 anos',
    '31 - 39 anos',
    '40 - 50 anos',
    '51 - 60 anos',
    '61+'
]

df['Faixa_Etaria'] = pd.cut(df['Idade_Usuario'], bins=faixas_definidas, labels=faixas_texto, right=True)

faixas_agrupadas = df.groupby('Faixa_Etaria',observed=True).agg(
    total=('contador', 'count'),
    media_duracao_minutos=('Duracao_Minutos', 'mean')
).reset_index()

faixas_agrupadas['media_duracao_minutos']=faixas_agrupadas['media_duracao_minutos'].round(2)

faixa_com_maior_recorrencia = faixas_agrupadas.loc[faixas_agrupadas['total'].idxmax()]


mensagem = (
    f"A faixa de idade com maior recorrência de aluguel é {faixa_com_maior_recorrencia['Faixa_Etaria']} "
    f"com um total de {faixa_com_maior_recorrencia['total']} aluguéis. "
    f"O tempo médio de aluguel é {faixa_com_maior_recorrencia['media_duracao_minutos']:.2f} minutos."
)

print(faixa_com_maior_recorrencia)

# faixa_com_maior_recorrencia.to_frame().T.to_csv('faixa_com_maior_recorrencia.csv',index=False)

Faixa_Etaria             31 - 39 anos
total                          226198
media_duracao_minutos           11.68
Name: 3, dtype: object


### 5.Quais são as estações com maior número de bikes alugadas/devolvidas? 

In [ ]:
alugueis_por_estacao = df.groupby('Estacao_Aluguel').agg(
    total_alugueis=('Estacao_Aluguel', 'count')
).reset_index()

devolucoes_por_estacao = df.groupby('Estacao_Chegada').agg(
    total_devolucoes=('Estacao_Chegada', 'count')
).reset_index()

estacoes_aluguel_devolucao = pd.merge(
    alugueis_por_estacao, devolucoes_por_estacao,
    left_on='Estacao_Aluguel', right_on='Estacao_Chegada',
    how='outer'
).fillna(0)

estacoes_aluguel_devolucao['total_alugueis'] = estacoes_aluguel_devolucao['total_alugueis'].astype(int)
estacoes_aluguel_devolucao['total_devolucoes'] = estacoes_aluguel_devolucao['total_devolucoes'].astype(int)

estacoes_aluguel_devolucao.rename(columns={'Estacao_Aluguel': 'Estacao'}, inplace=True)
estacoes_aluguel_devolucao.drop(columns=['Estacao_Chegada'], inplace=True)



estacoes_top_aluguel_devolucao = estacoes_aluguel_devolucao.sort_values(by=['total_alugueis', 'total_devolucoes'], ascending=[False, False]).head(5)

print(estacoes_top_aluguel_devolucao)

# estacoes_top_aluguel_devolucao.to_csv('estacoes_top_aluguel_devolucao.csv', index=False)


    Estacao  total_alugueis  total_devolucoes
0         1            6311              6115
189      27            6241              6220
191     271            5265              4917
441      64            4834              5165
347      41            4633              3836
